In [7]:
import pandas as pd
import numpy as np

from scipy import stats

import warnings
warnings.filterwarnings('ignore')

## 6-1 카이제곱검정 : 카이제곱분포
### 적합성 검정 : 다항 모집단 비율 차이
- 적어도 기대도수 5 이상일 때.
- H0 : 구해진 도수분포의 도수와 이론 도수가 차이가 없다
- H0 : 구해진 도수분포의 도수와 이론 도수가 차이가 있다

In [2]:
# 세 후보의 지지도가 다르다고 할 수 있을까
data = np.array([60, 50 ,40])

m0 = data.mean(); alpha = 0.05;
df = len(data)-1
chistat = sum((data - m0)**2/m0)
sp = 1 - stats.chi2.cdf(chistat, df)
cv = stats.chi2.ppf(1-alpha/2, df)

print("[수기 검정]")
print(f" 오른쪽 검정의 임계값: {cv:.3f}, 검정통계량: {chistat:.3f}")
print(f" 유의수준: {alpha:.3f}, 유의확률: {sp:.3f}")

print("[라이브러리 검정]")
stat, p = stats.chisquare(data, m0)
print(f" 검정 통계량{stat:.3f}, p-value {p:.3f}")

[수기 검정]
 오른쪽 검정의 임계값: 7.378, 검정통계량: 4.000
 유의수준: 0.050, 유의확률: 0.135
[라이브러리 검정]
 검정 통계량4.000, p-value 0.135


### 독립성 검정 : 한 모집단 내 여러 수준의 차이
- 교차분석, 다수의 인자들에 의해 분할되어 있는 데이터에서 인자들이 관찰값에 영향을 주고 있는지 여부 검정
- 검정조건
    - 자유도가 1인 경우, 전체 데이터 수가 30보다 크면서 각 칸의 빈도가 5이상
    - 데이터 수가 30보다 크면서 5 미만의 기대빈도의 칸이 전체 칸의 20%보다 적고 모든 칸에 1 이상의 기대빈도가 있다면 척도에 관계없이 사용 가능
    - 각 칸의 기대빈도가 5 미만인 경우 변수들의 범주를 묶거나 이항검정법 사용
    - 도수가 작아도 피셔의 정확검정을 이용하면 집계표의 독립성 검정 가능
- 가설 설정
    - H0 : 두 인자는 독립이다(연관이 없다)
    - H1 : 두 인자는 독립이 아니다(연관이 있다)

In [4]:
table = pd.DataFrame({
    '성별' : ['남자', '여자'], '안경O' : [10, 30], '안경X' : [40,20]
}).set_index('성별')
alpha = 0.05

ttl = table.sum().sum()
exp = []
r = table.sum(axis=1).values
c = table.sum(axis=0).values
for R in r:
    for C in c:
        exp.append(R*C/ttl)
print(' 기대값\n', np.array(exp).reshape(table.shape))
obs = table.values.ravel()
print(" 관찰값\n", np.array(obs).reshape(table.shape))

chistat = np.sum((obs-exp)**2/exp)
df = (table.shape[0]-1) * (table.shape[1]-1)
sp = 1 - stats.chi2.cdf(chistat, df)
cv = stats.chi2.ppf(1-alpha, df)
print("[수기 검정]")
print(f" 오른쪽 검정의 임계값: {cv:.3f}, 검정통계량: {chistat:.3f}")
print(f" 유의수준: {alpha:.3f}, 유의확률: {sp:.3f}")

print("[라이브러리 검정]")
chi2, p, df, expec = stats.chi2_contingency(table, correction=False)
print(f" 검정 통계량{ chi2:.3f}, p-value {p:.3f}, 기대값:\n {expec}")

 기대값
 [[20. 30.]
 [20. 30.]]
 관찰값
 [[10 40]
 [30 20]]
[수기 검정]
 오른쪽 검정의 임계값: 3.841, 검정통계량: 16.667
 유의수준: 0.050, 유의확률: 0.000
[라이브러리 검정]
 검정 통계량16.667, p-value 0.000, 기대값:
 [[20. 30.]
 [20. 30.]]


In [5]:
# 참고 Fisher's exact test
table = pd.DataFrame([[10, 2], [3, 5]], index=['A', 'B'], columns = ['승', '패'])
print("[데이터 확인]\n", table)

print("[라이브러리 검정]")
stats, p = stats.fisher_exact(table, alternative='greater')
print(f"검정통계량 {stats:.3f}, p-value {p:.3f}\n")

[데이터 확인]
     승  패
A  10  2
B   3  5
[라이브러리 검정]
검정통계량 8.333, p-value 0.052



### 동질성 검정 : 여러 모집단 간 여러 수준에 대한 차이
- 속성 A,B를 가진 부모집단들로부터 정해진 표본 크기 만큼 자료를 추출하는 경우 분할표에서 부모 집단의 비율이 동일한지 여부 검정
- H0 : 모든 집단의 분포가 차이가 없다
- H1 : 적어도 한 집단은 분포 상 서로 차이가 있다

In [9]:
table = pd.DataFrame({
    'TV' : ['A', 'B', 'C'], '청년' : [120, 30, 50], '중년' : [10, 75, 15], '장년':[10, 30, 60]
}).set_index('TV')
alpha = 0.05

ttl = table.sum().sum()
exp = []
r = table.sum(axis=1).values
c = table.sum(axis=0).values
for R in r:
    for C in c:
        exp.append(R*C/ttl)
print(" 기대값\n", np.array(exp).reshape(table.shape))        
obs = table.values.ravel()
print(" 관찰값\n", np.array(obs).reshape(table.shape))        

chistat = np.sum((obs - exp)**2 / exp)
df = (table.shape[0]-1)*(table.shape[1]-1)
sp = 1 - stats.chi2.cdf(chistat, df)
cv = stats.chi2.ppf(1-alpha, df)
print("[수기 검정]")
print(f" 오른쪽 검정의 임계값: {cv:.3f}, 검정통계량: {chistat:.3f}")
print(f" 유의수준: {alpha:.3f}, 유의확률: {sp:.3f}")

print("[라이브러리 검정]")
chi2, p, df, expec = stats.chi2_contingency(table, correction=False)
print(f" 검정 통계량 {chi2:.3f}, p-value {p:.3f}, \n기대값:\n {expec}")

 기대값
 [[70.   35.   35.  ]
 [67.5  33.75 33.75]
 [62.5  31.25 31.25]]
 관찰값
 [[120  10  10]
 [ 30  75  30]
 [ 50  15  60]]
[수기 검정]
 오른쪽 검정의 임계값: 9.488, 검정통계량: 180.495
 유의수준: 0.050, 유의확률: 0.000
[라이브러리 검정]
 검정 통계량 180.495, p-value 0.000, 
기대값:
 [[70.   35.   35.  ]
 [67.5  33.75 33.75]
 [62.5  31.25 31.25]]


## 6-2 Run 검정 : Run 검정표, Z분포
### 일표본 Run 검정
- 한 개의 샘플이 무작위로 추출되었는지 여부 검정
    - 런 : 동일한 관측값이 연속으로 이어진 것. 한 종류의 부호 혹은 집단이 시작하여 끝날 때까지의 한 덩어리
- 범주형 데이터의 경우, 각 범주의 개수와 런 개수를 사용하여 검정을 진행하고, 수치형 데이터는 중앙값을 기준으로 데이터 이진화 한 후 검정 진행
- H0 : 샘플이 무작위로 추출되었다.
- H1 : 샘플이 무작위로 추출되지 않았다.

In [25]:
from collections import Counter
# run을 세는 함수 정의
def count_run(data):
    count_run = 1
    for i, element in enumerate(data[:-1]):
        if element == data[i+1]:
            continue
        else:
            count_run += 1
    return count_run

data = ['a', 'a', 'b', 'b', 'a', 'a', 'a', 'a', 'b', 'b', 'b', 'b', 'a', 'b', 'a', 'a'
       'b', 'b', 'a', 'a', 'b', 'b', 'b', 'a', 'a', 'b', 'b', 'a', 'b', 'a']
data2 = list(map(lambda x: 0 if x == 'a' else 1, data))
a, b = Counter(data2)[0], Counter(data2)[1]
print(f"[데이터 확인]\n{Counter(data2)}")

print("[수기 검정]")
n1, n2 = a,b #n1, n2가 충분히 클 때 점근적 정규분포를 따름
run = count_run(data2)
avg = 2 * n1 * n2 / (n1+n2)+1
var = 2*n1*n2*(2*n1*n2-n1-n2)/((n1+n2)**2)/(n1+n2-1)

# 런의 총 갯수에 따라서 h 값을 정함
if run < 2*n1*n2/(n1+n2)+0.5:
    h = 0.5
else:
    h = -0.5
    
# n1, n2의 크기에 따라서 Z통계량 공식을 정함
if n1<=20 or n2<=20:
    Z = (run-avg+h) / np.sqrt(var) # 소표본
else:
    Z = (run-avg) / np.sqrt(var) # 대표본
sp = (1 - stats.norm.cdf(np.abs(Z)))*2
print(f" 기대값: {avg:.3f}, 분산: {var:.3f}, 검정통계량: {Z:.3f}, pvalue: {sp:.3f}")

print("[라이브러리 검정]")
from statsmodels.sandbox.stats.runs import runstest_1samp
zstat, pval = runstest_1samp(data2, cutoff = 'mean', correction=True)
print(f" 검정통계량, {zstat:.3f}, pvalue : {pval:.3f}")

[데이터 확인]
Counter({1: 15, 0: 14})
[수기 검정]
 기대값: 15.483, 분산: 6.974, 검정통계량: -0.372, pvalue: 0.710
[라이브러리 검정]
 검정통계량, 0.007, pvalue : 0.995


In [28]:
## 수치형 데이터의 경우
from collections import Counter
# run을 세는 함수 정의
def count_run(data):
    count_run = 1
    for i, element in enumerate(data[:-1]):
        if element == data[i+1]:
            continue
        else:
            count_run += 1
    return count_run

data = [50, 60, 70, 40, 30, 20, 10, 70, 80, 100]
median = np.median(data)
data2 = []
for sample in data:
    if sample >= median:
        data2.append(1)
    else:
        data2.append(0)
print(f"[데이터 확인]\n{Counter(data2)}")      

n1 = Counter(data2)[0] ; n2 = Counter(data2)[1]
run = count_run(data2)
avg = 2*n1*n2/(n1+n2)+1
var = 2*n1*n2*(2*n1*n2-n1-n2)/((n1+n2)**2)/(n1+n2-1)

# run 갯수에 따라 h값 정함
if run < 2*n1*n2/(n1+n2)+0.5:
    h = 0.5
else:
    h = -0.5

# n1, n2의 크기에 따라서 z통계량 공식을 정함
if n1 <= 20 or n2 < 20:
    Z = (run-avg+h) / np.sqrt(var)
else:
    Z = (run-avg) / np.sqrt(var)
sp = (1-stats.norm.cdf(np.abs(Z)))*2
print("[수기 검정]")
print(f" 기대값 {avg:.3f}, var: {var:.3f}, zstat: {Z:.3f}, pvalue : {sp:.3f}")

from statsmodels.sandbox.stats.runs import runstest_1samp
zstat, pval = runstest_1samp(data2, cutoff='median')
print("[라이브러리 검정]")
print(f" 검정통계량 {zstat:.3f}, pvalue {pval:.3f}")

[데이터 확인]
Counter({0: 5, 1: 5})
[수기 검정]
 기대값 6.000, var: 2.222, zstat: -1.006, pvalue : 0.314
[라이브러리 검정]
 검정통계량 -1.006, pvalue 0.314


### 이표본 Run 검정
- H0 : 두 데이터는 같은 분포에서 왔다
- H1 : 두 데이터는 다른 분포에서 왔다

In [32]:
# 수치형 데이터의 경우
# run을 세는 함수 정의
def count_run(data):
    count_run = 1
    for i, element in enumerate(data[:-1]):
        if element == data[i+1]:
            continue
        else:
            count_run += 1
    return count_run

data1 = [23, 42, 36, 27, 48, 52, 35, 31]
data2 = [43, 56, 38, 20, 46, 51, 36]

data1 = list(map(lambda x: float(x), data1))
data2 = list(map(lambda x: float(x), data2))
print(f"[데이터 확인]\n {Counter(data1)} \n {Counter(data2)}")

data = data1 + data2
median = np.median(data)
data3 = [1 if i >= median else 0 for i in data] # 중앙값 기준 이진화
print(f"[데이터 확인]\n {Counter(data3)}")

n1 = Counter(data3)[0]; n2 = Counter(data3)[1]
run = count_run(data3)
avg = 2*n1*n2 / (n1+n2)+1
var = 2*n1*n2*(2*n1*n2-n1-n2)/((n1+n2)**2) / (n1+n2-1)

if run < 2*n1*n2/(n1+n2)+0.5:
    h = 0.5
else:
    h = -0.5
    
if n1 <= 20 or n2 <= 20:
    Z = (run-avg+h) / np.sqrt(var)
else:
    Z = (run-avg) / (np.sqrt(var))
sp = (1 - stats.norm.cdf(np.abs(Z)))*2
    
print("[수기 검정]")
print(f" 기대값 {avg:.3f}, var: {var:.3f}, zstat: {Z:.3f}, pvalue : {sp:.3f}")

print("[라이브러리 검정]")
from statsmodels.sandbox.stats.runs import runstest_2samp
zstat, pval = runstest_2samp(data1, data2)
print(f" 검정통계량 {zstat:.3f}, pvalue {pval:.3f}")

[데이터 확인]
 Counter({23.0: 1, 42.0: 1, 36.0: 1, 27.0: 1, 48.0: 1, 52.0: 1, 35.0: 1, 31.0: 1}) 
 Counter({43.0: 1, 56.0: 1, 38.0: 1, 20.0: 1, 46.0: 1, 51.0: 1, 36.0: 1})
[데이터 확인]
 Counter({1: 8, 0: 7})
[수기 검정]
 기대값 8.467, var: 3.449, zstat: 0.018, pvalue : 0.986
[라이브러리 검정]
ties detected
 검정통계량 0.018, pvalue 0.986


## 6-3 이항변수 데이터 검정: 카이제곱 분포
### 맥니머 검정 (McNemar's test)
- 이항변수인 두 변수의 대응관계(paired)가 있는 데이터 분포의 차이
- H0 : 두 변수의 데이터 분포는 차이가 없다
- H1 : 두 변수의 데이터 분포는 차이가 있다    

In [33]:
# 프로모션 전후 상품에 대한 흥미 유무
table = pd.DataFrame([[9,12], [24,35]], index=['전_있음', '전_없음'], columns=['후_있음', '후_없음'])
print(f"[Data check]\n {table}")

# H0 : 두 경우 분포가 같다
# H1 : 두 경우 분포가 다르다
print("[수기 검정]") # 변화가 있는 b,c에 주목하고 변화가 없는 대각선은 무시
b = table.values[0][1]
c = table.values[1][0]
stat = (b-c)**2 / (b+c)
alpha = 0.05
df = 1
pval = 1 - stats.chi2.cdf(stat, df)
print(f" 검정통계량: {stat:.3f}, pvalue {pval:.3f}")

from statsmodels.stats.contingency_tables import mcnemar
# exact 값이 True이면 이항분포, False면 카이제곱분포 사용
print("[Library Test]")
mc = mcnemar(table.values, exact=False, correction=False)
print(f" stat {mc.statistic}, pvalue {mc.pvalue:.3f}")

[Data check]
       후_있음  후_없음
전_있음     9    12
전_없음    24    35
[수기 검정]
 검정통계량: 4.000, pvalue 0.046
[Library Test]
 stat 4.0, pvalue 0.046


### 코크란 Q 검정
- 이항변수인 세 변수 이상의 대응관계가 있는 데이터 분포 검정
- 대응관계 일원배치 분산분석의 비모수 버전
- H1: 적어도 한 쌍의 변수의 데이터 분포는 차이가 있다.

In [37]:
# 연예인 3명에 대한 호감도 비율 차이?
from scipy.stats import chi2
table = pd.DataFrame([
    [0,1,0,1,0,0,0,0,], [1,1,0,1,0,0,1,1], [0,1,1,1,1,1,1,1]], index=['가수1', '가수2', '가수3'],
    columns=[1,2,3,4,5,6,7,8]
).T
print(f"Check Data\n{table}")

# H1 : 적어도 한 쌍의 연예인은 호감도 비율이 차이가 있따
print("[Direct Test]")
n = table.shape[0]; k = table.shape[1]; df = n-1;
k_sums = np.array(table.sum(axis=0)); n_sums = np.array(table.sum(axis=1))
Q1 = k * sum(k_sums**2) - sum(k_sums)**2
Q2 = k * sum(n_sums**2) - sum(n_sums)
stat = df * Q1 / Q2
pval = 1 - stats.chi2.cdf(stat, df)
print(f"stat {stat:.3f}, pvalue {pval:.3f}")

print("[Library Test]")
from statsmodels.stats.contingency_tables import cochrans_q
ccq = cochrans_q(table)
print(f" stats {ccq.statistic:.3f}, pvalue {ccq.pvalue:.3f}")

# 모든 쌍의 맥니머 검정을 통해 가수1-가수3 간 호감도 차이 확인
from itertools import combinations
col_comp = list(combinations(table.columns, 2))

from statsmodels.stats.contingency_tables import mcnemar
for s1, s2 in col_comp:
    ct = pd.crosstab(table.loc[:, s1], table.loc[:, s2])
    mc = mcnemar(ct.values, exact=False, correction=False)
    stat, p = mc.statistic, mc.pvalue
    msg = f"{s1}-{s2}: stats {stat:.3f}, pvalue {p:.3f}"
    if p < 0.05:
        print(msg + "***")
    else:
        print(msg)

Check Data
   가수1  가수2  가수3
1    0    1    0
2    1    1    1
3    0    0    1
4    1    1    1
5    0    0    1
6    0    0    1
7    0    1    1
8    0    1    1
[Direct Test]
stat 3.500, pvalue 0.835
[Library Test]
 stats 6.333, pvalue 0.042
가수1-가수2: stats 3.000, pvalue 0.083
가수1-가수3: stats 5.000, pvalue 0.025***
가수2-가수3: stats 1.000, pvalue 0.317


## 6-4 부호, 순위 데이터검정
### 일표본 부호 검정: 이항분포, Z분포
- 일표본 t 검정 대응 : n 100 이하 시, 이항분포 & 100 이상 시, 정규분포

In [45]:
# data median M0 = 200 Test
# H1 : data median is different from 200

print("[Direct Test]")
data = np.array([203, 204, 197, 195, 201, 205, 198, 199, 194, 207])
M0 = 200

df = pd.DataFrame(data-M0, columns=['d'])
df = df.query("d != 0")
plus = len(df.query("d > 0"))
minus = len(df.query("d < 0"))
n = len(df) # num of effective data 
print(f" Effective number of data {n} num of plus(stat) {plus}")

# 검정통계량 B(plus 부호 갯수)는 B(n, p=0.5)인 이항분포를 따름
p = 0.5
result = pd.DataFrame()
for i in range(0, n+1):
    result.loc[i, 'X'] = i
    result.loc[i, 'prob'] = stats.binom.pmf(i, n, p)
result['value'] = result['X'] * result['prob']
print(f"[binom dist. based] prob dist. table:\n", result)

mean = result.sum()['value']
var = sum((result['X']-mean)**2 * result['prob'])
print(f" expectation {mean:.3f}, var {var:.3f} (= {n*p:.3f}, {n*p*(1-p):.3f})")

alpha = 0.05
start = int(stats.binom.pmf(alpha/2, n, p))
end = int(stats.binom.pmf(1-alpha/2, n, p))
print(f" stat: {plus}, critival value: {start}, {end}")

# 정규분포 근사
mean2 = n/2; var2 = n/4; s = np.sqrt(var2)
zstat = (plus-mean2)/s
ways = 'two'
if ways == 'two':
    sp = (1-stats.norm.cdf(np.abs(zstat)))*2
    cv = stats.norm.ppf(1-alpha/2)
    cv = f"+/-{cv:.3f}"
elif ways == 'one-right':
    sp = (1-stats.norm.cdf(zstat))
    cv = stats.norm.ppf(1-alpha)
    cv = f"{cv:.3f}"
elif ways == 'one-left':
    sp = (stats.norm.cdf(zstat))
    cv = stats.norm.ppf(alpha/2)
    cv = f"{cv:.3f}"
    
print(f"[Normal dist.] expectation {mean2:.3f}, var {var2:.3f}")
print(f" stat: {zstat:.3f}, critival value: {cv}")
print(f" sig standard: {alpha:.3f}, sigp prob: {sp:.3f}")

[Direct Test]
 Effective number of data 10 num of plus(stat) 5
[binom dist. based] prob dist. table:
        X      prob     value
0    0.0  0.000977  0.000000
1    1.0  0.009766  0.009766
2    2.0  0.043945  0.087891
3    3.0  0.117188  0.351562
4    4.0  0.205078  0.820312
5    5.0  0.246094  1.230469
6    6.0  0.205078  1.230469
7    7.0  0.117187  0.820312
8    8.0  0.043945  0.351562
9    9.0  0.009766  0.087891
10  10.0  0.000977  0.009766
 expectation 5.000, var 2.500 (= 5.000, 2.500)
 stat: 5, critival value: 0, 0
[Normal dist.] expectation 5.000, var 2.500
 stat: 0.000, critival value: +/-1.960
 sig standard: 0.050, sigp prob: 1.000


### 이표본 부호 검정: 이항분포, Z분포
- n이 100 이하일 경우, 부호검정 통계량은 이항분포를 따르고, 100 이상이면 정규분포를 따름
- 대응표본 t검정에 대응

In [48]:
# H1: A의 만족도가 더 높다
print("[Direct Test]")
data1 = np.array([4,3,5,2,1,3,4,3])
data2 = np.array([3,2,3,1,2,2,2,2])
M0 = 0

df = pd.DataFrame(data1-data2, columns=['d'])
df = df.query("d != 0")
plus = len(df.query("d > 0"))
minus = len(df.query("d < 0"))
n = len(df) # num of effective data 
print(f" Effective number of data {n} num of plus(stat) {plus}")

# 검정통계량 B(plus 부호 갯수)는 B(n, p=0.5)인 이항분포를 따름
p = 0.5
result = pd.DataFrame([])
for i in range(0, n+1):
    result.loc[i, 'X'] = i
    result.loc[i, 'prob'] = stats.binom.pmf(i, n, p)
result['value'] = result['X'] * result['prob']
print(f"[binom dist. based] prob dist. table:\n", result)

mean = result.sum()['value']
var = sum((result['X']-mean)**2 * result['prob'])
print(f" expectation {mean:.3f}, var {var:.3f} (= {n*p:.3f}, {n*p*(1-p):.3f})")

alpha = 0.05
start = int(stats.binom.pmf(alpha/2, n, p))
end = int(stats.binom.pmf(1-alpha/2, n, p))
print(f" stat: {plus}, critival value: {start}, {end}")

# 정규분포 근사
mean2 = n/2; var2 = n/4; s = np.sqrt(var2)
zstat = (plus-mean2)/s
ways = 'one-right'
if ways == 'two':
    sp = (1-stats.norm.cdf(np.abs(zstat)))*2
    cv = stats.norm.ppf(1-alpha/2)
    cv = f"+/-{cv:.3f}"
elif ways == 'one-right':
    sp = (1-stats.norm.cdf(zstat))
    cv = stats.norm.ppf(1-alpha)
    cv = f"{cv:.3f}"
elif ways == 'one-left':
    sp = (stats.norm.cdf(zstat))
    cv = stats.norm.ppf(alpha/2)
    cv = f"{cv:.3f}"
    
print(f"[Normal dist.] expectation {mean2:.3f}, var {var2:.3f}")
print(f" stat: {zstat:.3f}, critival value: {cv}")
print(f" sig standard: {alpha:.3f}, sigp prob: {sp:.3f}")

[Direct Test]
 Effective number of data 8 num of plus(stat) 7
[binom dist. based] prob dist. table:
      X      prob    value
0  0.0  0.003906  0.00000
1  1.0  0.031250  0.03125
2  2.0  0.109375  0.21875
3  3.0  0.218750  0.65625
4  4.0  0.273437  1.09375
5  5.0  0.218750  1.09375
6  6.0  0.109375  0.65625
7  7.0  0.031250  0.21875
8  8.0  0.003906  0.03125
 expectation 4.000, var 2.000 (= 4.000, 2.000)
 stat: 7, critival value: 0, 0
[Normal dist.] expectation 4.000, var 2.000
 stat: 2.121, critival value: 1.645
 sig standard: 0.050, sigp prob: 0.017


### 일표본 윌콕슨 부호순위 검정: 윌콕슨 부호순위 검정표, Z분포
- n이 20이하이면 윌콕슨 순위합 분포를 다르고, 20이상이면 정규분포 근사
- 일표본 t검정에 대응

In [51]:
# 데이터의 중앙값으로 알려진 M0=200 가설 검정
print("[Direct Test]")
data = np.array([203, 204, 197, 195, 201, 205, 198, 199, 194, 207])
M0 = 200
d = data - M0
table = pd.DataFrame(d, columns=['d'])
table = table.query("d != 0")
alpha = 0.05
n = len(table)
table['sign'] = np.sign(table['d'])
table['abs_d'] = table['d'].abs()
table['rank'] = table['abs_d'].rank(method='average')
print(" sign rank cal table:\n ", table)

plus = table.query('sign==1')['rank'].sum()
minus = table.query('sign==-1')['rank'].sum()
stat = np.minimum(plus, minus)
print("[Wilcoxon sign rank dist. based]")
print(f" Eff data n : {n}, stat: {stat}")

print('[Library Test]')
stat, p = stats.wilcoxon([M0 for i in range(len(data))], data)
print(f" stat: {stat:.3f}, pvalue: {p:.3f}")

# 정규 근사
wstat = np.minimum(plus, minus) # 작은 쪽이 검정통계량
mean = n*(n+1)/4; var = n*(n+1)*(2*n+1)/24
s = np.sqrt(var)
zstat = (wstat-mean)/s
ways = 'two'
if ways == 'two':
    sp = (1 - stats.norm.cdf(np.abs(zstat)))*2
    cv = stats.norm.ppf(1-alpha/2)
    cv = f'+/-{cv:.3f}'
elif ways == 'one-right':
    sp = 1 - stats.norm.cdf(zstat)
    cv = stats.norm.ppf(1-alpha)
    cv = f"{cv:.3f}"
elif ways == 'one-left':
    sp = stats.norm.cdf(zstat)
    cv = stats.norm.ppf(alpha)
    cv = f"{cv:.3f}"
    
print(f"[Normal Dist based] Exp;{mean:.3f}, Var:{var:.3f}")
print(f" stat {zstat:.3f}, critival value {cv:}")
print(f" sig standard {alpha:.3f}, sig prob. {sp:.3f}")

[Direct Test]
 sign rank cal table:
     d  sign  abs_d  rank
0  3     1      3   4.5
1  4     1      4   6.0
2 -3    -1      3   4.5
3 -5    -1      5   7.5
4  1     1      1   1.5
5  5     1      5   7.5
6 -2    -1      2   3.0
7 -1    -1      1   1.5
8 -6    -1      6   9.0
9  7     1      7  10.0
[Wilcoxon sign rank dist. based]
 Eff data n : 10, stat: 25.5
[Library Test]
 stat: 25.500, pvalue: 0.846
[Normal Dist based] Exp;27.500, Var:96.250
 stat -0.204, critival value +/-1.960
 sig standard 0.050, sig prob. 0.838


### 이표본 윌콕슨 부호순위 검정 : 윌콕슨 부호순위 검정표, Z분포
- 데이터 갯수가 20 이하이면 윌콕슨 순위합 분포, 20이상이면 정규분포 근사
- 대응표본 t검정 대응

In [53]:
# 동일한 피험자 8명에게 맥박 2번 측정 시 차이 유무 검정
print("[Direct Test]")
data1 = np.array([79, 96, 85, 69, 88, 75, 83, 88])
data2 = np.array([70, 88, 73, 74, 75, 79, 77, 81])
d = data1 - data2
table = pd.DataFrame(d, columns=['d'])
table = table.query("d != 0")
n = len(table)
table['sign'] = np.sign(table['d'])
table['abs_d'] = table['d'].abs()
table['rank'] = table['abs_d'].rank(method='average')
print( "부호순위 계산표:\n", table)

plus = table.query('sign==1')['rank'].sum()
plus = table.query('sign==-1')['rank'].sum()
stat = np.minimum(plus, minus)
print("[Wilcoxon sign-rank dist. based]")
print(f" Eff. data num: {n}, stat: {stat}")

print("[Library test]")
stat, p = stats.wilcoxon(data1, data2, zero_method='wilcox')
print(f" stat: {stat:.3f}, pvalue {p:.3f}")

# 정규 근사
wstat = np.minimum(plus, minus)
mean = n*(n+1)/4; var = n*(n+1)*(2*n+1)/24
s = np.sqrt(var); zstat = (wstat-mean)/s
ways = 'two'
if ways == 'two':
    sp = (1-stats.norm.cdf(np.abs(zstat)))*2
    cv = stats.norm.ppf(1-alpha/2)
    cv = f"+/-{cv:.3f}"
elif ways == 'one-right':
    sp = (1-stats.norm.cdf(zstat))
    cv = stats.norm.ppf(1-alpha)
    cv = f"{cv:.3f}"
elif ways == 'one-left':
    sp = (stats.norm.cdf(zstat))
    cv = stats.norm.ppf(alpha/2)
    cv = f"{cv:.3f}"
    
print(f"[Normal dist.] expectation {mean:.3f}, var {var:.3f}")
print(f" stat: {zstat:.3f}, critival value: {cv}")
print(f" sig standard: {alpha:.3f}, sigp prob: {sp:.3f}")

[Direct Test]
부호순위 계산표:
     d  sign  abs_d  rank
0   9     1      9   6.0
1   8     1      8   5.0
2  12     1     12   7.0
3  -5    -1      5   2.0
4  13     1     13   8.0
5  -4    -1      4   1.0
6   6     1      6   3.0
7   7     1      7   4.0
[Wilcoxon sign-rank dist. based]
 Eff. data num: 8, stat: 3.0
[Library test]
 stat: 3.000, pvalue 0.039
[Normal dist.] expectation 18.000, var 51.000
 stat: -2.100, critival value: +/-1.960
 sig standard: 0.050, sigp prob: 0.036


### 윌콕슨 순위합 검정(만 휘트니 U 검정) : 윌콕슨 순위합 검정표, Z분포
- 데이터 합이 25 이하이면 순위합 분포, 25 이상이면 정규분포 근사
- 독립표본 t 검정 대응

In [55]:
# 월 별 영업성적 차이 검정 (중앙값 차이)
# H0 : 두 중앙값 차이는 0이다

data1 = [87, 75, 65, 95, 90, 81, 93]
data2 = [57, 85, 90, 83, 87, 71]
n1 = len(data1); n2 = len(data2)
M1 = np.median(data1); M2 = np.median(data2)

print(f"[Direct Test]")
table1 = pd.DataFrame(data1, columns=['data']);
table1['M'] = M1
table2 = pd.DataFrame(data2, columns=['data']);
table2['M'] = M2
table = pd.concat([table1, table2])
table['rank'] = table['data'].rank(method='average')
print(f" 순위합 계산표:\n", table)

W1 = table.query("M==@M1")['rank'].sum()
W2 = table.query("M==@M2")['rank'].sum()
U1 = W1 - n1*(n1+1)/2
U2 = W2 - n2*(n2+1)/2
print(f" n of data: {n1}, {n2}")
print(f" ranksum stat: {W1} or {W2}")
print(f" Mann Whitney stat: {U1} or {U2}")

print("[Library Test]")
zstat, p = stats.ranksums(data1, data2, alternative='two-sided')
print(f" ranksum norm dist. stats: {zstat:.3f}, pvalue: {p:.3f}")
mstat, p = stats.mannwhitneyu(data1, data2, alternative='two-sided')
print(f" Manwhitneyu stat {mstat:.3f}, pvalue {p:.3f}")      

# 정규 근사
mean = n1*n2/2; var = n1*n2*(n1+n2+1)/12; s = np.sqrt(var)
Z1 = (U1 - mean)/s ; Z2 = (U2 - mean)/s
alpha = 0.05
zstat = Z1
ways = 'two'
if ways == 'two':
    sp = (1-stats.norm.cdf(np.abs(zstat)))*2
    cv = stats.norm.ppf(1-alpha/2)
    cv = f"+/-{cv:.3f}"
elif ways == 'one-right':
    sp = (1-stats.norm.cdf(zstat))
    cv = stats.norm.ppf(1-alpha)
    cv = f"{cv:.3f}"
elif ways == 'one-left':
    sp = (stats.norm.cdf(zstat))
    cv = stats.norm.ppf(alpha/2)
    cv = f"{cv:.3f}"
    
print(f"[Normal dist.] expectation {mean:.3f}, var {var:.3f}")
print(f" stat: {zstat:.3f}, critival value: {cv}")
print(f" sig standard: {alpha:.3f}, sigp prob: {sp:.3f}")

[Direct Test]
 순위합 계산표:
    data     M  rank
0    87  87.0   8.5
1    75  87.0   4.0
2    65  87.0   2.0
3    95  87.0  13.0
4    90  87.0  10.5
5    81  87.0   5.0
6    93  87.0  12.0
0    57  84.0   1.0
1    85  84.0   7.0
2    90  84.0  10.5
3    83  84.0   6.0
4    87  84.0   8.5
5    71  84.0   3.0
 n of data: 7, 6
 ranksum stat: 55.0 or 36.0
 Mann Whitney stat: 27.0 or 15.0
[Library Test]
 ranksum norm dist. stats: 0.857, pvalue: 0.391
 Manwhitneyu stat 27.000, pvalue 0.431
[Normal dist.] expectation 21.000, var 49.000
 stat: 0.857, critival value: +/-1.960
 sig standard: 0.050, sigp prob: 0.391
